In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

In [3]:
train1 = train.drop(['Descript', 'Resolution', 'Address'], axis = 1)
train1.head()

,Dates,Category,DayOfWeek,PdDistrict,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,-122.438738,37.771541


In [4]:
train1 = train1.query('Y != 90')

In [5]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

# Ad_dict = {'.*?ST / .*?ST':0, '.*?AV / .*?AV':1, '.*?ST / .*?AV':2, '.*?AV / .*?ST':3, '.*?ST':4, '.*?AV':5}
# train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)
# train1['Address'].unique()
# Ad_dict = {'.* / .*':0, '.*':1}
# train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# # DayOfWeek...label, PdDistrict...count

,Category,DayOfWeek,PdDistrict,X,Y,Date,Year,Month,Day,Hour
0,37,6,105284,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,6,105284,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,6,105284,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,6,105284,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,6,49311,-122.438738,37.771541,2015-05-13,2015,5,13,23


In [7]:
test1 = test.drop(['Address'], axis = 1)
test1.head()

,Id,Dates,DayOfWeek,PdDistrict,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,-122.437394,37.721412


In [8]:
Dt = pd.to_datetime(test1['Dates'])
test1['Date'] = Dt.dt.date
test1['Year'] = Dt.dt.year
test1['Month'] = Dt.dt.month
test1['Day'] = Dt.dt.day
test1['Hour'] = Dt.dt.hour
test1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test1['DayOfWeek'] = le.fit_transform(test1['DayOfWeek'])

count_encoding = test1['PdDistrict'].value_counts().to_dict()
test1['PdDistrict'] = test1['PdDistrict'].map(count_encoding)

# Ad_dict = {'.* / .*':0, '.*':1}
# test1['Address'] = test1['Address'].replace(Ad_dict,regex = True)
test1.head()

,Id,DayOfWeek,PdDistrict,X,Y,Date,Year,Month,Day,Hour
0,0,3,89591,-122.399588,37.735051,2015-05-10,2015,5,10,23
1,1,3,89591,-122.391523,37.732432,2015-05-10,2015,5,10,23
2,2,3,107017,-122.426002,37.792212,2015-05-10,2015,5,10,23
3,3,3,80084,-122.437394,37.721412,2015-05-10,2015,5,10,23
4,4,3,80084,-122.437394,37.721412,2015-05-10,2015,5,10,23


In [10]:
cols = ['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test1[cols]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [12]:
y_val.head()

483661    21
728982     4
842807     7
73242     21
280700    16
Name: Category, dtype: int32

In [13]:
from sklearn.ensemble import RandomForestClassifier as RFC

# rf = RFC(n_jobs=-1,          # 複数のCPUコアを使って並列に学習します。-1は最大値。
#           random_state=2525)  # 乱数のシードです。
rf = RFC(n_estimators=60, max_depth=32, random_state=1219)

rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=32, n_estimators=60, random_state=1219)

In [14]:
y_pred = rf.predict(X_val)

In [15]:
# y_val.reshape(-1,1)

In [16]:
# print(f"acc: {rf.score(y_val, y_pred)}")

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.29265551737884166

In [18]:
rf_submission = rf.predict(X_test)

KeyboardInterrupt: 

In [ ]:
import csv
pred = rf.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [ ]:
submission.to_csv("rf1.csv", index=False)